In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
df_dirty_train = pd.read_csv('../data/train.csv')
df_dirty_test = pd.read_csv('../data/test.csv')
df_dirty_train.head()

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price
0,122881,hdb flat for sale in 866 yishun street 81,sembawang / yishun (d27),866 yishun street 81,hdb 4 rooms,NaN,1988.0,3.0,2.0,1115,...,unspecified,NaN,116.0,https://www.99.co/singapore/hdb/866-yishun-str...,1.414399,103.837196,0,yishun south,yishun,514500.0
1,259374,hdb flat for sale in 506b serangoon north aven...,hougang / punggol / sengkang (d19),hdb-serangoon estate,hdb,99-year leasehold,1992.0,4.0,2.0,1575,...,unspecified,"1, 2, 3, 4, 5, 6 br",NaN,https://www.99.co/singapore/hdb/hdbserangoon-e...,1.372597,103.875625,0,serangoon north,serangoon,995400.0
2,665422,4 bed condo for sale in meyerhouse,128 meyer road,meyerhouse,condo,freehold,2022.0,4.0,6.0,3070,...,partial,"studio, 3, 4, 5, 6 br",56.0,https://www.99.co/singapore/condos-apartments/...,1.298773,103.895798,0,mountbatten,marine parade,8485000.0
3,857699,3 bed condo for sale in leedon green,26 leedon heights,leedon green,Condo,freehold,2023.0,3.0,2.0,958,...,partial,"studio, 1, 2, 3, 4 br",638.0,https://www.99.co/singapore/condos-apartments/...,1.312364,103.803271,0,farrer court,bukit timah,2626000.0
4,216061,2 bed condo for sale in one bernam,1 bernam street,one bernam,condo,99-year leasehold,2026.0,2.0,1.0,732,...,unspecified,"studio, 1, 2, 3, 4, 5 br",351.0,https://www.99.co/singapore/condos-apartments/...,1.273959,103.843635,0,anson,downtown core,1764000.0


In [14]:
#a.	Remove duplicates and invalid data
print(df_dirty_train.shape[0])
df_cleaned = df_dirty_train.drop_duplicates()
df_cleaned = df_cleaned[df_cleaned.size_sqft > 0]
df_cleaned = df_cleaned[df_cleaned.num_beds >= 0]
df_cleaned = df_cleaned[df_cleaned.num_baths >= 0]
df_cleaned = df_cleaned[df_cleaned.price > 0]



print(df_cleaned.shape[0])
print(f'Records dropped :{df_dirty_train.shape[0] - df_cleaned.shape[0]}' )


20254
19646
Records dropped :608


In [21]:
#b.	Remove irrelevant data
df_cleaned = df_cleaned.drop('property_details_url', axis = 1)
df_cleaned = df_cleaned.drop('listing_id', axis = 1)
df_cleaned.dropna(subset=['price'], inplace=True)


In [22]:
#c.	Standardize capitalization
df_cleaned.fillna({'built_year': 0}, inplace=True)
df_cleaned['built_year'] = df_cleaned['built_year'].astype(str).apply(lambda x: x.replace('.0',''))
df_cleaned['property_type'] = df_cleaned['property_type'].str.lower()
df_cleaned['tenure'] = df_cleaned['tenure'].str.lower()
df_cleaned['furnishing'] = df_cleaned['furnishing'].str.lower()
df_cleaned['subzone'] = df_cleaned['subzone'].str.lower()
df_cleaned['planning_area'] = df_cleaned['planning_area'].str.lower()

In [25]:
#d.	Convert data type
df_cleaned['built_year'] = df_cleaned['built_year'].astype(int)
df_cleaned['num_beds'] = df_cleaned['num_beds'].astype(str).apply(lambda x: x.replace('.0',''))
df_cleaned['num_baths'] = df_cleaned['num_baths'].astype(str).apply(lambda x: x.replace('.0',''))
df_cleaned['num_beds'] = df_cleaned['num_beds'].astype(int)
df_cleaned['num_baths'] = df_cleaned['num_baths'].astype(int)
df_cleaned.head()
df_cleaned.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19646 entries, 0 to 20253
Data columns (total 19 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   title                 19646 non-null  object 
 1   address               19646 non-null  object 
 2   property_name         19646 non-null  object 
 3   property_type         19646 non-null  object 
 4   tenure                17934 non-null  object 
 5   built_year            19646 non-null  int64  
 6   num_beds              19646 non-null  int64  
 7   num_baths             19646 non-null  int64  
 8   size_sqft             19646 non-null  int64  
 9   floor_level           3455 non-null   object 
 10  furnishing            19646 non-null  object 
 11  available_unit_types  18216 non-null  object 
 12  total_num_units       14019 non-null  float64
 13  lat                   19646 non-null  float64
 14  lng                   19646 non-null  float64
 15  elevation          

In [ ]:
#e.	Clear formatting

In [ ]:
#f.	Fix errors

In [ ]:
#g.	Language translation

In [ ]:
#h.	Handle missing values

In [ ]:
#No need to run this unless all data has been cleaned.
df_dirty_test.to_csv('../data/test_cleaned.csv')  
df_dirty_train.to_csv('../data/train_cleaned.csv')